### Path set up and import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from IPython.display import clear_output

!pip install pytest-runner --upgrade
!pip install ftfy

!pip install faiss-gpu
# !pip install pytorch==1.7.1 torchvision cudatoolkit==11.0
# !pip install ftfy regex tqdm
# !pip install git+https://github.com/openai/CLIP.git

import os
import numpy as np
import json
import torch
import sys
import random

import pandas as pd
import glob
import faiss
from PIL import Image
from pathlib import Path

clear_output()

In [3]:
CODE_PATH = Path('/content/drive/MyDrive/Sketch/Image Retrieval with Text and Sketch/code/')
MODEL_PATH = Path('/content/drive/MyDrive/Sketch/Image Retrieval with Text and Sketch/model/')
IMAGE_PATH = Path('/content/drive/MyDrive/KeyFrames_AIO_Pending/')
# SKETCH_PATH = Path('/content/drive/MyDrive/Sketch/Sketch Image/') ## Dùng cho phần lưu ảnh sketch cho query

In [4]:
model_config_file = CODE_PATH / 'training/model_configs/ViT-B-16.json'
model_file = MODEL_PATH / 'tsbir_model_final.pt'

In [5]:
sys.path.append(str(CODE_PATH))

##make sure CODE_PATH is pointing to the correct path containing clip.py before running
from clip.model import convert_weights, CLIP
from clip.clip import _transform, load

from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.utils.data.distributed import DistributedSampler
from dataclasses import dataclass

In [6]:
gpu = 0
torch.cuda.set_device(gpu)

with open(model_config_file, 'r') as f:
    model_info = json.load(f)

model = CLIP(**model_info)

loc = "cuda:{}".format(gpu)
checkpoint = torch.load(model_file, map_location=loc)

sd = checkpoint["state_dict"]
if next(iter(sd.items()))[0].startswith('module'):
    sd = {k[len('module.'):]: v for k, v in sd.items()}

model.load_state_dict(sd, strict=False)

model.eval()

model = model.cuda()

## Extract feature for all the images and save to numpy

In [7]:
@dataclass
class DataInfo:
    dataloader: DataLoader
    sampler: DistributedSampler

class SimpleImageFolder(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __getitem__(self, index):
        image_path = self.image_paths[index]

        x = Image.open(image_path)
        if self.transform is not None:
            x = self.transform(x)
        return x, image_path

    def __len__(self):
        return len(self.image_paths)

In [8]:
convert_weights(model)
# preprocess_train = _transform(model.visual.input_resolution, is_train=True)
preprocess_val = _transform(model.visual.input_resolution, is_train=False)
# preprocess_fn = (preprocess_train, preprocess_val)

model = model.cuda()

In [9]:
class FileSket:
    def __init__(self, db_path= IMAGE_PATH, quiet=False, num_batch = 1, model = model, batch_size = 32, transform = preprocess_val):
        self.db_path = db_path
        # self.csv_folder_path = os.path.join(db_path, 'map-keyframes')
        self.npy_folder_path = os.path.join(db_path, 'npy-sketch-keyframes')

        self.quiet = quiet

        self.model = model

        self.batch_size = batch_size
        self.transform = transform
        self.num_batch = num_batch

        if not os.path.exists(self.npy_folder_path):
            os.mkdir(self.npy_folder_path)

    def __get_group_id(self, video_name):
        # video_name = 'L01_V001'
        return video_name[:3] # L01


    def __get_group_path(self, group_id):
        for batch in self.num_batch:
            if os.path.exists(os.path.join(self.db_path, f'keyframes_batch{batch}', f'KeyFrames_{group_id}')):
                return os.path.join(self.db_path, f'keyframes_batch{batch}', f'KeyFrames_{group_id}')

    def __get_kf_folder(self,video_name):
        group_id = self.__get_group_id(video_name)
        group_folder = self.__get_group_path(group_id)
        return os.path.join(group_folder, video_name)

    def __collate_fn(self, batch):
        batch = list(filter(lambda x: x is not None, batch))
        return torch.utils.data.dataloader.default_collate(batch)

    # Hàm tạo một file npy theo tên video
    def __create_npy(self, video_name):
        kf_folder = self.__get_kf_folder(video_name)

        if not self.quiet:
            print(f'Folder path: {kf_folder}')

        # Kiểm tra thư mục có tồn tại hay không
        if not os.path.exists(kf_folder):
            raise Exception(f'Folder {video_name} is not exist!!!')

        img_paths = sorted(glob.glob(kf_folder+'/*.jpg'))

        if len(img_paths) == 0:
            raise Exception(f'Folder {video_name} is empty!!!')

        image_list = []
        for item in img_paths:
            if '.ipynb' not in str(item):
                image_list.append(str(item))

        dataset = SimpleImageFolder(image_list, transform = self.transform)
        dataloader = DataLoader(
            dataset,
            batch_size = self.batch_size,
            collate_fn = self.__collate_fn,
            shuffle=False,
            num_workers=1,
            pin_memory=True,
            sampler=None,
            drop_last=False,
        )
        dataloader.num_samples = len(dataset)
        dataloader.num_batches = len(dataloader)

        data = DataInfo(dataloader, None)

        all_image_path = []
        all_image_features = []
        batch_num = 0

        with torch.no_grad():
            for batch in dataloader:
                print('Batch: ' + str(batch_num), end='')
                images, image_paths = batch
                images = images.cuda(gpu, non_blocking=True)

                image_features = model.encode_image(images)

                image_features = image_features / image_features.norm(dim=-1, keepdim=True)
                for i in image_features:
                    all_image_features.append(i.cpu().numpy())
                for i in image_paths:
                    all_image_path.append(i)

                batch_num += 1

                print(' -- Done\n')

        file_path = f'{self.npy_folder_path}/{video_name}.npy'
        with open(file_path, 'wb') as f:
            np.save(f, np.array(all_image_features, dtype=np.float32))

        if not self.quiet:
            print(f'Creating {video_name}.npy successfully!!!')


    def create_video_npy(self, video_names):
        # video_names = self.__get_video_names()
        for name in video_names:
            self.__create_npy(name)
            # print(f'create_video_npy {name}') ### Checking


    def __get_video_names(self, group_id):
        paths = []
        for batch in self.num_batch:
            if os.path.exists(os.path.join(self.db_path, f'keyframes_batch{batch}', f'KeyFrames_{group_id}')):
                kf_folder_path = os.path.join(self.db_path, f'keyframes_batch{batch}', f'KeyFrames_{group_id}')
                paths += glob.glob(f'{kf_folder_path}/*')
        sorted(paths)
        # print(f'paths {paths}') ### Checking
        names = []
        for path in paths:
            name = os.path.split(path)[-1]
            # if len(name) == 8 and os.path.split(path)[-1][-4:] == 'V001': ### Checking
            if len(name) == 8:
                names.append(name)
        # print(f'names {names}') ### Checking
        return names

    def create_group_npy(self, group_ids):
        for id in group_ids:
            if not self.quiet:
                print(f'Starting create npy files for {id}')
            video_names = self.__get_video_names(id)
            self.create_video_npy(video_names)


    def create_all_npy(self):
        # Lấy groups id từ Keyframes
        group_paths = []
        for batch in self.num_batch:
            group_paths += glob.glob(f'{self.db_path}/keyframes_batch{batch}/KeyFrames_*')
        sorted(group_paths)
        # print(f'group_paths {group_paths}') ### Checking

        group_ids = []
        for path in group_paths:
            if len(path.split('/')[-1]) == 13:
                group_ids.append(path[-3:])
        sorted(group_ids)
        # print(f'group_ids {group_ids}') ### Checking
        self.create_group_npy(group_ids)


    def __read_all_npy(self):
        # Lấy tất cả đường dẫn file npy
        npy_paths = sorted(glob.glob(self.npy_folder_path+'/*.npy'))
        # print(f'__read_all_npy {npy_paths}') ### Checking

        # Xuất hiện thông báo lỗi không tìm thấy file
        if len(npy_paths) == 0:
            raise Exception('Error not found npy files!!!')
        return np.concatenate([np.load(path) for path in npy_paths], axis=0)


    def __create_faiss_bin(self):
        feats = self.__read_all_npy()
        index = faiss.IndexFlatIP(feats.shape[1])
        index.add(feats)
        bin_path = self.db_path+'/sketch_index.bin'
        faiss.write_index(index, bin_path)


    def create_bin(self):
        self.__create_faiss_bin()

In [10]:
# Tạo file numpy
num_batch = [6]
batch_size = 32
f4d = FileSket(db_path= IMAGE_PATH, quiet=False, num_batch = num_batch, batch_size = batch_size)
f4d.create_all_npy()

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.

Batch: 12 -- Done

Batch: 13 -- Done

Batch: 14 -- Done

Batch: 15 -- Done

Batch: 16 -- Done

Batch: 17 -- Done

Batch: 18 -- Done

Batch: 19 -- Done

Batch: 20 -- Done

Batch: 21 -- Done

Batch: 22 -- Done

Batch: 23 -- Done

Batch: 24 -- Done

Batch: 25 -- Done

Batch: 26 -- Done

Batch: 27 -- Done

Batch: 28 -- Done

Batch: 29 -- Done

Creating L35_V003.npy successfully!!!
Folder path: /content/drive/MyDrive/KeyFrames_AIO_Pending/keyframes_batch6/KeyFrames_L35/L35_V004
Batch: 0 -- Done

Batch: 1 -- Done

Batch: 2 -- Done

Batch: 3 -- Done

Batch: 4 -- Done

Batch: 5 -- Done

Batch: 6 -- Done

Batch: 7 -- Done

Batch: 8 -- Done

Batch: 9 -- Done

Batch: 10 -- Done

Batch: 11 -- Done

Batch: 12 -- Done

Batch: 13 -- Done

Batch: 14 -- Done

Batch: 15 -- Done

Batch: 16 -- Done

Batch: 17 -- Done

Batch: 18 -- Done

Batch: 19 -- Done

Batch: 20 -- Done

Batch: 21 -- Done

Batch: 22 -- Done

Batch: 23 -- Done

Batch: 24 -- Done
